In [8]:
epochs = 20
n_train_items = 6000
rounds = 200
n_workers = 10

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from torch.utils.tensorboard import SummaryWriter


In [3]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]


workers = connect_to_workers(n_workers=20)


In [9]:
class Arguments():
    def __init__(self):
        self.batch_size = 60
        self.test_batch_size = 60
        self.epochs = epochs
        self.rounds = rounds
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 2
        self.save_model = False
        self.n_train_items = n_train_items

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [4]:
train_MNIST =   datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
test_MNIST =     datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

In [7]:

# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.batch_size
# )
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
#                        transforms.ToTensor(),
#                        transforms.Normalize((0.1307,), (0.3081,))
#                    ])),
#     batch_size=args.test_batch_size
# )
def unblance_dataset(dataset):
    loader = []
    count = 0
    batch_contain_one_class = 0
    while(len(loader) < n_train_items):
        for (data, target) in dataset:
            if (len(loader)>=n_train_items):
                break
            if (target == batch_contain_one_class):
                loader.append((data,target))
                count +=1
                batch_contain_one_class = (count)//args.batch_size%10
                #print(batch_contain_one_class)
    return loader


#---
unbalance_train_dataset = unblance_dataset(train_MNIST)

#test
print(len(unbalance_train_dataset))
print(unbalance_train_dataset[740][1])

#put into dataloader
train_dataloader = torch.utils.data.DataLoader(unbalance_train_dataset, batch_size=args.test_batch_size)
test_dataloader = torch.utils.data.DataLoader(test_MNIST, batch_size=args.test_batch_size)

print(len(test_dataloader)*64)


6000
2
10688


In [11]:
len(list(test_dataloader.__iter__())[0][1])

60

In [12]:
# from PIL import Image
# import numpy 
# #mnist_dataset.__getitem__(2)[1]
# a = (mnist_dataset.__getitem__(0)[0]).numpy()
# a.dtype = 'uint8'
# print(a)
# Image.fromarray(a[0], mode= 'P')

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1)
        self.conv2 = nn.Conv2d(32, 64, 5, 1)
        self.fc1 = nn.Linear(4*4*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [15]:
def model_init(workers, Net):
    model_list = list()
    for worker in workers:
        model_list.append(Net)
    return model_list
def opt_init(model_list):
    opt_list = list()
    for model  in model_list:
        opt_list.append(optim.SGD(model.parameters(), lr=args.lr))
    return opt_list

In [18]:
def train(args, device, less_train_dataloader, opt_list, workers):
    global model_list
    ## start training and record the model into model_list
    
    for epoch in range(args.epochs):
        for batch_idx, (data, target) in enumerate(less_train_dataloader): # <-- now it is a distributed dataset
            model_on_worker = model_list[batch_idx%len(workers)]
            model_on_worker.train()
            model_on_worker.send(workers[batch_idx%len(workers)]) # <-- NEW: send the model to the right location

            data_on_worker = data.send(workers[batch_idx%len(workers)])
            target_on_worker = target.send(workers[batch_idx%len(workers)])

            data_on_worker, target_on_worker = data_on_worker.to(device), target_on_worker.to(device)

            opt_list[batch_idx%len(workers)].zero_grad()

            output = model_on_worker(data_on_worker)
            loss = F.nll_loss(output, target_on_worker)
            loss.backward()

            opt_list[batch_idx%len(workers)].step()
            model_on_worker.get() # <-- NEW: get the model back

            model_list[batch_idx%len(workers)] = model_on_worker #When len(dataloader) is longer than the len(worker) send and get must be modified
            #model_list here is full of the model which has trained on the workers, there are all different now.

        if epoch % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, n_train_items, n_train_items ,
                100. * epoch / args.epochs, loss.item()))


    ##Aggregation time
    new_model = []
    tmp_model = Net().to(device)
    with torch.no_grad():
        for p in model_list[0].parameters():
            new_model.append(0)
            
        for m in model_list:
            for par_idx, par in enumerate(m.parameters()):
                #average the model_list
                new_model[par_idx] = new_model[par_idx]+par.data
                # we get new model in list format and need to set_ to model
        for worker in range(len(workers)):
            for par_idx in range(len(new_model)):
                list(model_list[worker].parameters())[par_idx].set_(new_model[par_idx]/len(workers))
        #init model with new_model
        


In [32]:
def test(args, model, device, test_loader, r):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)*(args.batch_size)
    accuracy = 100. * correct / (len(test_loader)*args.batch_size)
    #Since the test loader here is a list, we can get the len by * it with batch.size
    
    
    writer.add_scalar('Accuracy', accuracy,r)
    writer.add_scalar('Loss', test_loss, r)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader)* (args.batch_size),
        accuracy))

In [33]:
logdir = '/root/notebooks/tensorflow/logs/pysyft_noniid6000'
writer = SummaryWriter(logdir)

In [34]:
%%time
#optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment


model_list = []
model_list = model_init(workers, Net().to(device))
opt_list = opt_init(model_list)
# not finish in train, finish latter
pars = [list(model.parameters()) for model in model_list]

for r in range(1, args.rounds + 1):
    train(args, device, train_dataloader, opt_list, workers)
    print("After training")
    test(args, model_list[0], device, test_dataloader, r)


if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 0 [6000/6000 (0%)]	Loss: 2.207905
Train Epoch: 2 [6000/6000 (10%)]	Loss: 0.819587
Train Epoch: 4 [6000/6000 (20%)]	Loss: 0.250379
Train Epoch: 6 [6000/6000 (30%)]	Loss: 0.127622
Train Epoch: 8 [6000/6000 (40%)]	Loss: 0.075499
Train Epoch: 10 [6000/6000 (50%)]	Loss: 0.047826
Train Epoch: 12 [6000/6000 (60%)]	Loss: 0.032459
Train Epoch: 14 [6000/6000 (70%)]	Loss: 0.023193
Train Epoch: 16 [6000/6000 (80%)]	Loss: 0.017442
Train Epoch: 18 [6000/6000 (90%)]	Loss: 0.013493
After training

Test set: Average loss: 0.0884, Accuracy: 9734/10020 (97%)

Train Epoch: 0 [6000/6000 (0%)]	Loss: 0.010714
Train Epoch: 2 [6000/6000 (10%)]	Loss: 0.008609
Train Epoch: 4 [6000/6000 (20%)]	Loss: 0.006932
Train Epoch: 6 [6000/6000 (30%)]	Loss: 0.005633
Train Epoch: 8 [6000/6000 (40%)]	Loss: 0.004655
Train Epoch: 10 [6000/6000 (50%)]	Loss: 0.003878
Train Epoch: 12 [6000/6000 (60%)]	Loss: 0.003252
Train Epoch: 14 [6000/6000 (70%)]	Loss: 0.002716
Train Epoch: 16 [6000/6000 (80%)]	Loss: 0.002332
Train